# RKI-Daten aufbereiten
Corona-Datensatz des RKI lesen, filtern, aufsummieren...

Für das Seminar "Datenjournalismus" im Sommersemester 2022

Hochschule Mainz, Studiengang Digital Media, 6. Semester

CC-BY Jan Eggers

## Vorbereitung: Bibliotheken laden
Zusatzpakete statten Python mit Fähigkeiten zur Ein- und Ausgabe aus. Sie müssen in der jeweiligen Python-Umgebung installiert sein - wenn sie das nicht sind, muss man einmal auf die Kommandozeile und beispielsweise mit dem Befehl
```conda install pandas```
das Pandas-Paket installieren, das wir für Tabellen ("Dataframes") brauchen. 

(Wer Updates für ein Paket einspielen will, nutzt dazu den Befehl ```conda update ...``` oder einfach  ```conda update --all```)

In [ ]:
import pandas as pd
import numpy as np

Den obigen Textblock ausgeführt - und nichts ist passiert? Dann hat alles geklappt - und wir können loslegen. 

## CSV-Datensatz vom RKI einlesen
Wir lesen die CSV-Datei des RKI aus: 

Die Datei
* https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland

laden wir und schauen sie uns danach kurz an.

Die Datei ist ein Feld-Wald-und-Wiesen-CSV: 
- Als Trennzeichen wird das Komma (",") verwendet, das keine Probleme mit Kommazahlen bereitet
- Text ist in Anführungszeichen - und er enthält auch keine Steuerzeichen für Zeilensprünge, die gerne für Chaos sorgen
- Der Zeichensatz der Datei ist UTF-8 - im Universal-Format gibt es keine Probleme mit Umlauten etc. 

In [ ]:
rki_url = "https://media.githubusercontent.com/media/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/master/Aktuell_Deutschland_SarsCov2_Infektionen.csv"

rki_df = pd.read_csv(rki_url)
rki_df.head(5)

** Beschreiben **

Was ist im Dataframe eigentlich drin?


In [ ]:
rki_df.describe()

**Filtern**

Nur das Bundesland oder, alternativ, nur einen Kreis.

In [ ]:
#rki_hessen_df = rki_df.query("Bundesland == 'Hessen'")
rki_hessen_df = rki_df.loc[(rki_df["IdLandkreis"] >= 6000) & (rki_df["IdLandkreis"]< 7000)]
rki_hessen_df.head(5)

Etwas nacharbeiten: 
- Die Spalten `Meldedatum` und `RefDatum` in ordentliche Datumsangaben umformatieren
- Die Spalte `IdLandkreis` von einer Zahl in einen String umwandeln - mit führender Null

In [ ]:
from datetime import date, timedelta
from dateutil.parser import parse

for i in rki_hessen_df.index:
    rki_hessen_df.loc[i,'IdLandkreis'] = "0"+str(rki_hessen_df.IdLandkreis[i])
    rki_hessen_df.loc[i,'Meldedatum'] = parse(rki_hessen_df.Meldedatum[i]).date()
    rki_hessen_df.loc[i,'Refdatum'] = parse(rki_hessen_df.Refdatum[i]).date()
    
rki_hessen_df.head(5)

**Tabelle pivotieren**: Hessische Fälle nehmen, Neufälle filtern (Variable NeuerFall muss 0 oder 1 sein), nach Meldedatum aufsummieren (das Argument `aggfunc=np.sum` bestimmt, dass als Funktion zum Aggregieren die Summe der Werte berechnet wird)

In [ ]:
mein_df = rki_hessen_df.query("NeuerFall in  (0,1)")
neufaelle_df = pd.pivot_table(mein_df,index=["Meldedatum"],values=["AnzahlFall"],aggfunc=np.sum)
neufaelle_df.head(5)

**Nach Altersgruppen aufsummieren** - ein Pivot nach Meldedatum über die Altersgruppen als Spalten, den Summen (aggfunc) der Fallzahlen. 

In [ ]:
datum = date.today() - timedelta(days=7)
datum

In [ ]:
mein_df = rki_hessen_df.loc[rki_hessen_df["Meldedatum"] >= datum]
faelle_nach_alter_df = pd.pivot_table(mein_df,index=["Meldedatum"],columns=["Altersgruppe"],values=["AnzahlFall"],aggfunc=np.sum,fill_value = 0)
faelle_nach_alter_df.head(5)

## Tabelle als Excel-Datei ausgeben
Und jetzt hätten wir das Ganze gerne wieder als Excel-Datei. Das ist zum Glück einfach.

Die Parameter sagen: Spaltennamen mit in die Tabelle schreiben, auch den Index (das Datum). Und Zahlen mit Komma ausgeben! Alle Formatierungsbefehle für `to_excel` finden sich [in der Pandas-Dokumentation.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html) 

In [ ]:
faelle_nach_alter_df.to_excel ('export.xlsx', header=True)

**Nur Mut! Man kann nichts kaputt machen!**